In [13]:
from pulp import *
from itertools import chain, combinations
%run pravljenjeGrafova.ipynb


# funkcija za podskupove
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


# funkcija za susjede
def neighbors(x):
    n = []
    for (i,j) in Eweak:
        if i==x and Eweak[i,j] == 1:
            n.append(j)
    return n


# susjedstvo
def create_adjacency_dict(graph):
    adjacency_dict = {}
    
    for vertex, neighbors in graph.items():
        for neighbor in neighbors:
            if int(vertex)!=int(neighbor):
                adjacency_dict[(int(vertex), int(neighbor))] = 1
    
    for vertex in graph:
        for other_vertex in graph:
            if (int(vertex), int(other_vertex)) not in adjacency_dict and int(vertex)!=int(other_vertex) :
                adjacency_dict[(int(vertex), int(other_vertex))] = 0
    
    return adjacency_dict
            
    

{'1': ['6', '9', '10', '13', '14', '15', '18', '19', '20', '21', '24', '25', '26', '27', '28'], '2': ['5', '8', '10', '12', '14', '15', '17', '19', '20', '21', '23', '25', '26', '27', '28'], '3': ['4', '7', '10', '11', '14', '15', '16', '19', '20', '21', '22', '25', '26', '27', '28'], '4': ['3', '8', '9', '12', '13', '15', '17', '18', '20', '21', '23', '24', '26', '27', '28'], '5': ['2', '7', '9', '11', '13', '15', '16', '18', '20', '21', '22', '24', '26', '27', '28'], '6': ['1', '7', '8', '11', '12', '15', '16', '17', '20', '21', '22', '23', '26', '27', '28'], '7': ['3', '5', '6', '12', '13', '14', '17', '18', '19', '21', '23', '24', '25', '27', '28'], '8': ['2', '4', '6', '11', '13', '14', '16', '18', '19', '21', '22', '24', '25', '27', '28'], '9': ['1', '4', '5', '11', '12', '14', '16', '17', '19', '21', '22', '23', '25', '27', '28'], '10': ['1', '2', '3', '11', '12', '13', '16', '17', '18', '21', '22', '23', '24', '27', '28'], '11': ['3', '5', '6', '8', '9', '10', '17', '18', '19',

In [14]:
g = napraviGraf("graf-14.txt")

dictE= create_adjacency_dict(g)
Eweak = dict(sorted(dictE.items()))

V=[*range(1,len(g)+1,1)]

print(Eweak)


{(1, 2): 1, (1, 3): 1, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 0, (1, 8): 0, (1, 9): 0, (1, 10): 0, (1, 11): 0, (1, 12): 0, (1, 13): 1, (1, 14): 1, (2, 1): 1, (2, 3): 1, (2, 4): 1, (2, 5): 1, (2, 6): 1, (2, 7): 1, (2, 8): 1, (2, 9): 0, (2, 10): 0, (2, 11): 0, (2, 12): 0, (2, 13): 0, (2, 14): 0, (3, 1): 1, (3, 2): 1, (3, 4): 0, (3, 5): 0, (3, 6): 1, (3, 7): 1, (3, 8): 0, (3, 9): 0, (3, 10): 0, (3, 11): 0, (3, 12): 0, (3, 13): 0, (3, 14): 0, (4, 1): 0, (4, 2): 1, (4, 3): 0, (4, 5): 1, (4, 6): 0, (4, 7): 0, (4, 8): 0, (4, 9): 0, (4, 10): 0, (4, 11): 0, (4, 12): 0, (4, 13): 0, (4, 14): 0, (5, 1): 0, (5, 2): 1, (5, 3): 0, (5, 4): 1, (5, 6): 0, (5, 7): 0, (5, 8): 1, (5, 9): 0, (5, 10): 0, (5, 11): 0, (5, 12): 0, (5, 13): 0, (5, 14): 0, (6, 1): 0, (6, 2): 1, (6, 3): 1, (6, 4): 0, (6, 5): 0, (6, 7): 1, (6, 8): 0, (6, 9): 1, (6, 10): 0, (6, 11): 0, (6, 12): 0, (6, 13): 0, (6, 14): 0, (7, 1): 0, (7, 2): 1, (7, 3): 1, (7, 4): 0, (7, 5): 0, (7, 6): 1, (7, 8): 1, (7, 9): 0, (7, 10): 0, (7, 11): 0,

In [15]:

# model - problem minimizacije
model = LpProblem("MinimumWeaklyConnectedDominatingSet", LpMinimize)

# varijable
x = LpVariable.dicts("x", Eweak, 0, 1, LpBinary)
y = LpVariable.dicts("y", V , 0, 1, LpBinary)

# funkcija cilja
model += 1 + lpSum( x [ (u, v) ]  for (u, v) in Eweak )

# ogranicenja 1, 2 i 3
for u in V:
    for v in V:
        if u!=v:
            model += x[ (u, v) ] <= Eweak[u,v]
            model += x[ (u, v) ] == x[ (v, u) ]
            model += y[u] + y[v] >= 2*x[ (u, v) ]

# ogranicenje 4
for S in powerset(V):
    for u in S:
        for v in S:
            if u!=v and len(S)>2:
                model += lpSum( x[ (u, v) ] ) <= len(S)-1

# ogranicenje 5
for u in V:
    for v in V:
        if u!=v:
            model += lpSum( x[ (u,v) ]) == lpSum( y[t] for t in V) - 1

# ogranicenje 6
for v in V:
    for t in neighbors(v): 
        model += lpSum( y[t] ) >= 1


# rjesavanje
model.solve()

# ispis DS
selected = [u for u, var in y.items() if var.varValue == 1]
print("Selected:", selected)





Selected: [2, 3, 10, 13]


KeyboardInterrupt: 